In [ ]:

import functions_framework
import time
import random
from EmbeddingPredictionClient import EmbeddingPredictionClient  
from google.cloud import bigquery
import json
import asyncio

async def exponential_backoff_retries(client, text=None, image_file=None, max_retries=5, embedding_type=None):
    """
    This function applies exponential backoff with retries to the API calls.
    """
    attempt = 0
    while attempt < max_retries:
        try:
            # Try to get the embedding from the client
            if embedding_type=="multimodal_embedding":
                    return client.get_multimodal_embedding(text, image_file)
            elif embedding_type=="text_embedding":
                    return client.get_text_embedding(text)
        except Exception as e:
            attempt += 1
            backoff_delay = min(2 ** attempt + random.uniform(0, 1), 32)  # Exponential backoff with jitter
            print(f"Attempt {attempt} failed with error {e}. Retrying in {backoff_delay:.2f} seconds...")
            time.sleep(backoff_delay)  # Wait before retrying

    raise Exception("Max retries reached. Could not complete the request.")

    
async def generate_query_embedding(client,text=None,image_file=None, embedding_type=None):
    try:
        # Retry logic with exponential backoff to calculate query embeddings
        result = exponential_backoff_retries(embedding_client, text, image_file, embedding_type)
        
        # Respond with the successful embedding response
        return {
            "text_embedding": result.text_embedding,
            "image_embedding": result.image_embedding
        }, 200

    except Exception as e:
        # Handle failure after max retries
        return f"Error: {str(e)}", 500


async def get_media_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50, query_filter=""):
    """Query nearest neighbors using cosine similarity in BigQuery for multimodal embeddings."""
    
    # Record the start time
    start_time = time.time()
    options="""'{"fraction_lists_to_search": 0.5}'"""
    #options="""'{"use_brute_force":true}' """

    sql = f"""  
         WITH search_results AS
         (
              SELECT
              search_results.base.uri as fileUri,  
              search_results.base.combined_multimodal_id as unique_id,
              search_results.distance,  -- The computed distance (similarity score) between the embeddings
              search_results.base.asset_id ,
              search_results.base.ml_generate_embedding_start_sec as startOffset_seconds,
              search_results.base.ml_generate_embedding_end_sec as endOffset_seconds,  
              search_results.base.content_type as asset_type,
              ROW_NUMBER() OVER (PARTITION BY search_results.base.asset_id ORDER BY distance) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT * FROM  `{dataset}.{table}` WHERE 1=1 {query_filter}), --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                
              ) search_results
              
          )
          -- Step 2: Aggregate relevance per document  
            ,aggregated_results AS (
                SELECT
                    asset_id,
                    COUNT(*) AS chunk_count,  -- The number of chunks for this document
                    SUM(distance) AS total_distance,  -- Sum of the distances for this document's chunks
                    AVG(distance) AS avg_distance  -- Alternatively, you can use the average distance
                FROM search_results
                GROUP BY asset_id
            ),

            -- Step 3: Rank the documents by relevance (number of chunks and sum of distances)
            ranked_documents AS (
                SELECT
                    asset_id,
                    chunk_count,
                    total_distance,
                    avg_distance,
                    ROW_NUMBER() OVER (ORDER BY chunk_count DESC, total_distance ASC) AS final_rank  -- Rank by chunk_count and then distance

                FROM aggregated_results
            )

            -- Step 4: Retrieve the top-k ranked documents based on relevance
            SELECT * FROM (
              SELECT  
                sr.asset_id,  
                sr.fileUri,  
                sr.asset_type,
                ROW_NUMBER() OVER (PARTITION BY SR.asset_id) AS IDX,
                STRING_AGG(CONCAT("""+"'{startOffset_seconds:', sr.startOffset_seconds, ',endOffset_seconds:', sr.endOffset_seconds, '}')"""+f""", ", " ) 
                OVER (PARTITION BY sr.asset_id ORDER BY sr.startOffset_seconds) AS time_lines
               -- sr.distance,
               -- final_rank--,
               -- rank_within_document
            FROM search_results sr
            JOIN ranked_documents rd ON sr.asset_id = rd.asset_id
            WHERE rd.final_rank <= {top_k} -- Return the top-k documents based on chunk relevance
            ORDER BY rd.final_rank, sr.rank_within_document  -- Order by document relevance and chunk rank
            )
            WHERE IDX=1
    """       
    #print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()
    
    output=[]
    for row in results:
        output.append({'asset_id':row['asset_id'], 'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"]})

    
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

async def get_content_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50):
    """Query nearest neighbors using cosine similarity in BigQuery for text embeddings."""
    
    # Record the start time
    start_time = time.time()
    options="""'{"fraction_lists_to_search": 1}'"""
    #options="""'{"use_brute_force":true}' """

    sql = f"""  
         WITH search_results AS
         (
              SELECT
              search_results.base.content as content,  
              search_results.base.combined_id as combined_id,
              search_results.base.unique_id,
              distance,  -- The computed distance (similarity score) between the embeddings
              search_results.base.asset_id,
              search_results.base.headline,
              ifnull(search_results.base.html_safe_text,search_results.base.description) as description,
              search_results.base.startOffset_seconds,
              search_results.base.endOffset_seconds,
              search_results.base.fileUri,
              search_results.base.asset_type,
              ROW_NUMBER() OVER (PARTITION BY  search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                TABLE `{dataset}.{table}`, --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                   
              ) search_results              
          )
          -- Step 2: Aggregate relevance per document (original_document_id)
            ,aggregated_results AS (
                SELECT
                    asset_id,        
                    MIN(distance) AS min_distance  -- Alternatively, you can use the average distance
                FROM search_results
                GROUP BY asset_id
            ),

            -- Step 3: Rank the documents by relevance (number of chunks and sum of distances)
            ranked_documents AS (
                SELECT
                    asset_id,
                    min_distance,
                    ROW_NUMBER() OVER (ORDER BY min_distance ASC) AS final_rank  -- Rank by chunk_count and then distance
                FROM aggregated_results
            )

            -- Step 4: Retrieve the top-k ranked documents based on relevance
            SELECT * FROM (
              SELECT  
                sr.asset_id,  
                sr.headline,
                sr.description,
                sr.combined_id,
                sr.unique_id,
                sr.fileUri,
                sr.asset_type,
                ROW_NUMBER() OVER (PARTITION BY SR.asset_id) AS IDX,
                STRING_AGG(CONCAT("""+"'{startOffset_seconds:', sr.startOffset_seconds, ',endOffset_seconds:', sr.endOffset_seconds, '}')"""+f""", ", " ) 
                OVER (PARTITION BY sr.asset_id ORDER BY sr.startOffset_seconds) AS time_lines
                --sr.distance,
                --final_rank--,
               -- rank_within_document
            FROM search_results sr
            JOIN ranked_documents rd ON sr.asset_id = rd.asset_id
            WHERE rd.final_rank <= {top_k} -- Return the top-k documents based on chunk relevance      
            --and sr.asset_id like '%00261507986b0faf31c775597d2d24beb4381e43%'
            ORDER BY rd.final_rank, sr.rank_within_document  -- Order by document relevance and chunk rank
            )
            WHERE IDX=1
    """       
    print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()
    
    output=[]
    for row in results:
        output.append({'asset_id':row['asset_id'], 'headline':row['headline'],'description':row['description'],'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"]})

    
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

def merge_result(combined_list):
    # Step 2: Create a dictionary to merge by 'id'
    merged_dict = {}

    # Step 3: Iterate through the combined list and merge dictionaries by 'id'
    for d in combined_list:
        id_value = d['asset_id']

        # If the id already exists in merged_dict, update it
        if id_value in merged_dict:
            merged_dict[id_value].update(d)
        else:
            # If the id doesn't exist, add the dictionary as it is
            merged_dict[id_value] = d.copy()

    # Step 4: Convert the merged dictionary back into a list
    final_merged_list = list(merged_dict.values())
    
    return final_merged_list



async def get_nearest_contet(request):
    """
    Cloud Function entry point. This function handles the incoming request, 
    performs exponential backoff retries, and returns the embedding response.
    """ 
    # Parse the incoming request to extract text or image file
    request_json = request.get_json(silent=True)
    text = request_json.get('search_query')
    image_file = request_json.get('image_file')  # Assume it's the path or base64 string of the image
    project_id = request_json.get('project')  
    region = request_json.get('region')  
    
    # Load configuration from config.json
    with open('config.json') as config_file:
         config = json.load(config_file)
    
    
    top_k=int(config['top_k'])  
    dataset= config['dataset']
    content_table=config['content_table']
    mm_table=config['mm_table']
    content_source_embedding_column=config['content_source_embedding_column']
    mm_source_embedding_column=config['mm_source_embedding_column'] 
    if image_file=="" or image_file=="None":
        image_file=None
#     project_id='nine-quality-test'
#     region="us-central1"
#     text='curtis sittenfeld'
#     image_file=None
    
#     top_k=50     
#     dataset='vlt_media_embeddings_integration'
#     content_table='vlt_all_media_content_text_embeddings'
#     mm_table='vlt_imgvdo_multimodal_embeddings'
#     content_source_embedding_column='text_embedding_result'
#     mm_source_embedding_column='ml_generate_embedding_result'

    # Initialize the EmbeddingPredictionClient outside the function for reuse
    embedding_client = EmbeddingPredictionClient(project=project_id , location=region,api_regional_endpoint=region+"-aiplatform.googleapis.com")
        
    if not text and not image_file:
        print('you are here')
        return 'Error: At least one of "text" or "image_file" must be provided.', 400
     
    content_result=[]
    #media_text_result=[]
    media_image_result=[]
    if text:
        #if a text is given, calculate both multiomdal embedding and text embedding of the search query
        txtembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='text_embedding'))
        mmembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='multimodal_embedding')) 
        txtembding_for_text_result=txtembding_for_text_result .text_embedding
        mmembding_for_text_result=mmembding_for_text_result.text_embedding
        #find nearest neighbours both from text embedding and multimodal embedding
        content_result = await asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k))
        #media_text_result = await asyncio.create_task(get_media_nearest_neighbors(mmembding_for_text_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k))
               
    if image_file:
        #if an image is given convert image to 64bytestring and extract embedding
        mmembding_for_image_result = await asyncio.create_task(exponential_backoff_retries(embedding_client, image_file, embedding_type='multimodal_embedding'))
        mmembding_for_image_result=mmembding_for_text_result.image_embedding
        #find nearest neighbours both from multimodal embedding
        media_image_result = await asyncio.create_task(get_media_nearest_neighbors(mmembding_for_image_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k))
        media_image_result=media_image_result
        
    final_merged_list=merge_result(content_result+media_image_result)
    
    return final_merged_list#, content_result, media_text_result, media_image_result


@functions_framework.http
async def search_content_function(request):
 
    result = await get_nearest_contet(request) 
    return result#[0],result[1],result[2]

# @functions_framework.http
# def search_content_function(request):
#     """This is the entry point for the Cloud Function."""
#     try:
#         loop = asyncio.get_event_loop()
#     except RuntimeError as e:
#         # If no event loop is running, create a new event loop for this thread
#         loop = asyncio.new_event_loop()
#         asyncio.set_event_loop(loop)
#     result = loop.run_until_complete(get_nearest_contet(request))
#     return result
     

In [88]:
with open("data/girl.jpeg", "rb") as image_file:
    image_byte_string = image_file.read()

In [40]:
type(image_byte_string)

bytes

In [53]:
from unittest.mock import Mock
import json

image=b'\x89\x50\x4e\x47\x0d\x0a\x1a\x0a\x00\x00\x00\x0d\x49\x48\x44\x52\x00\x00\x00\x30\x00\x00\x00\x42\x08\x06\x00\x00\x00\x11\xe8\x55\x4d\x00\x00\x00\x01\x73\x52\x47\x42\x00\xae\xce\x1c\xe9\x00\x00\x00\x04\x67\x41\x4d\x41\x00\x00\xb1\x8f\x0b\xfc\x61\x05\x00\x00\x00\x09\x70\x48\x59\x73\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7\x6f\xa8\x64\x00\x00\x04\xd3\x49\x44\x41\x54\x68\x43\xed\x9a\x4d\x4c\xdb\x66\x18\xc7\xff\x8e\xed\x7c\x38\x21\x10\xb2\xad\x1b\x2d\xd0\x42\x37\xc4\x3e\xba\x6a\x55\xa7\x56\x82\x4a\x2b\xea\x26\x55\x48\x54\x3b\x8c\xc3\x24\x34\x4d\xdb\x91\x03\x27\x2e\x1c\xd6\xcb\x2e\x3b\x20\x76\x42\x4c\x3d\xa0\x6a\x07\x90\x36\x36\x69\x07\xb4\x4e\x1b\xe7\x7d\x08\x6d\xad\xd8\x54\x46\x17\xca\x04\x6c\x40\x9c\x90\xc4\x21\x71\xec\x77\xcf\x1b\xac\x55\x5a\x4b\x06\x8e\x83\x8b\xe4\x9f\x64\xe1\xd8\x8e\xfd\xfc\xde\xf7\x79\x1e\x5b\x31\x42\x22\x91\x60\x38\xc2\xf8\xac\xbf\x47\x16\x4f\xe0\x51\xe8\xba\x8e\x85\x85\x05\x18\x86\x61\x6d\xa9\x1d\x35\x13\x98\x9c\x9c\x44\xa1\x50\xb0\xb6\xd4\x0e\xaf\x88\x1f\x45\x26\x93\xc1\xf0\xf0\xf0\xbf\x33\xb0\xb2\xb2\x82\xd1\xd1\x51\xa8\xaa\x8a\xf1\xf1\x71\xf4\xf5\xf5\x61\x6c\x6c\x0c\xb9\x5c\xae\xbc\xbf\x1a\x6a\x56\xc4\x53\x53\x53\xd6\xda\x2e\x3c\xe0\x89\x89\x09\x0c\x0c\x0c\x60\x7a\x7a\x1a\xf9\x7c\x1e\x23\x23\x23\x55\xd7\xc9\xa1\x75\xa1\xde\xde\x5e\x0c\x0d\x0d\x41\x51\x14\x04\x02\x01\x0c\x0e\x0e\x62\x66\x66\x06\xa5\x52\xc9\x3a\xc2\x1e\x87\x26\x60\x9a\x26\x44\x51\xb4\x3e\x01\x92\x24\xa1\xab\xab\xcb\xfa\x64\x9f\x43\x15\xf8\x2f\xbc\x26\xaa\xe5\xd0\x04\x6a\x85\x27\xe0\x36\x35\x13\xe8\xef\xef\xb7\xd6\x76\xe9\xe8\xe8\xb0\xd6\x1e\xd0\xd3\xd3\x63\xad\xd9\xc7\xbb\x13\xbb\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\xd4\x4c\xc0\xa4\x53\xe7\x4d\x19\x29\x5d\x81\xaa\x87\x91\x35\x82\x30\xd8\x83\xdf\x46\x9d\xc2\xf1\x9f\x55\x8a\x4c\xc2\x46\x21\x8c\xd5\x1c\x2d\x9a\x8c\xfb\xdb\xc0\x6f\x1b\xc0\xf7\x9b\x0c\x6a\xd2\xc0\x6b\xc7\x25\x04\x64\xe0\xd2\x69\xe0\xd9\x58\x11\xcf\x28\x39\xc4\xe4\x3c\xfc\xbe\x12\x04\x1c\x3c\x14\x87\x04\x04\x1a\x5d\x1f\x36\x0b\x0a\xe6\x93\x8d\x98\xfd\x1d\xf8\x6a\xd5\x04\xb6\x18\x58\x56\x07\x68\x1f\x24\x01\xf1\xa8\x0f\xc9\x02\x6d\xe7\xaf\x04\xc2\x02\x9e\x8e\x89\xb8\xdc\x2a\xe0\xf5\x93\x06\x3a\xa3\x29\xd4\xfb\xf3\x90\x7c\xb4\x9f\xed\x3f\xa4\xaa\x05\x18\x05\xbf\x63\x4a\x58\xcc\xc6\xf0\xf1\x8f\x21\xcc\xf1\xc0\x69\xa4\x59\xc6\x80\x10\xa5\x94\x89\x4b\xe5\xa3\xb0\x43\x8b\x46\x0b\x05\xcf\xd2\x74\x0c\x09\x81\xbf\xdc\x08\x89\x10\x48\xe4\xcd\x17\x7d\x78\xf7\x79\x0d\xed\x11\x15\xb2\xb0\xff\x77\x06\x55\x09\xf0\xe0\x35\xca\xf3\xf9\xad\xa7\xf0\xce\xac\x0f\x6c\xad\x40\x39\x44\x27\x8d\xca\x78\xe3\x65\x11\x97\x9a\x81\xd6\x68\x89\xd2\x44\x43\x9c\x46\x57\x11\x8b\x48\xea\x21\x24\x8b\x41\xfc\x9a\x8a\xe0\xd6\x12\x30\x7b\x97\x64\x37\x69\x96\xfc\x22\x7c\xed\x32\x6e\x76\x97\x70\xa6\x61\x93\xbc\x8a\xfb\x4a\x29\xfb\x02\x02\x8d\xbc\x21\xe3\x67\xf5\x09\xbc\xfd\x39\x05\xbf\x41\xa3\x29\x99\x10\xda\x82\x18\xbd\x00\xbc\x12\x4f\xe1\xc9\x80\x06\x51\xa0\x72\x16\xb8\x2a\xbf\x0c\x2d\xf4\x3d\xd3\xf4\x95\x67\x6d\xb3\x10\xc2\x2f\xc9\x7a\x8c\xfd\x04\xdc\x5b\x24\x09\xda\x27\x44\x05\xdc\xb8\x26\xe0\xd5\xf8\x3a\x42\x54\x17\xe5\xef\x54\xc0\xa6\x80\x00\x9d\x3a\xca\x72\xae\x1e\xef\x7f\x1b\xc1\xfd\x79\x8d\xd2\xc0\x8f\x73\x9d\x12\xae\x5f\xc8\xa1\x39\x94\x41\x48\xd2\xa9\x0f\x51\xaa\xec\x09\x89\xd0\x52\x32\x45\x2c\xd1\x79\xde\x9b\x0b\x63\xfd\x0e\x49\x98\x3a\x84\x96\x10\xbe\xec\xdb\x41\x47\xdd\x16\x65\x5a\xe5\x77\x68\xb6\xda\x28\xbf\x30\x4f\x83\xcf\xee\x51\xf0\x77\x68\xaa\xeb\xfd\x68\x39\x29\xe2\xc3\x8b\xd9\x72\x0e\x2b\x52\xf1\x7f\x82\xe7\xb0\xf2\x31\xbc\xfb\xb4\x47\x52\xb8\x79\x39\x8b\xf3\xe7\xa8\x3d\xc9\x54\x33\x34\x9b\xa3\x54\x4f\x5b\x45\x85\x8e\xa8\x1c\xa2\x2d\x81\x22\x8d\xda\xdd\x74\x0c\x9f\xfc\x40\x41\x8a\x26\xda\xda\x24\x7c\xd4\x55\x42\x4b\x38\x4d\x05\xc8\xf5\x0e\x32\xa9\x0c\x7e\x1a\xe5\x13\x4a\x1a\x1f\x5c\xd4\x50\x77\x5a\x06\x2b\x9a\x98\x4b\x18\xb8\xad\x36\x96\xaf\x55\x09\x5b\x02\xdb\x7a\x10\xdf\x2c\x53\xde\xff\x5d\x80\x10\x97\x31\x78\x16\x78\xa1\x61\x83\x82\xe7\xd3\x6d\x23\x23\x2d\x89\x66\x92\xb8\x7e\x9e\xea\x80\x5a\x2c\x4b\x9a\xf8\xfa\x0f\x20\x4d\xd7\xe2\x15\xb4\x17\xb6\x04\xd6\xb4\x30\x3e\x5d\xe2\x81\xca\xe8\x6e\x15\x71\x26\x9e\x41\x70\x1f\x05\x57\x19\x46\x9d\x87\x3a\x50\x63\x06\xdd\x9d\x94\x4a\x34\xb9\xdf\xfd\xc9\xb0\xa2\x45\x9c\x17\x58\x4c\x05\x80\x14\xa5\x4a\x88\xe1\xea\x73\xd4\xea\x83\x1a\x6d\xad\x26\xf8\x5d\x78\x4d\x34\x51\x03\xb8\x72\x8a\x4a\x3c\x0c\xa8\xeb\x25\xac\x66\xfc\x30\x99\xc3\x02\xb7\xd7\x28\xdc\xbf\x8a\x74\x97\x05\x5e\x6a\xd4\xa0\xf8\xa8\xf9\x3b\x44\x80\x66\xb2\x25\x62\x42\x8e\x49\x60\x39\x03\x6b\x39\x3e\x34\x0e\x0b\x1c\x8b\x02\x6d\x67\x43\x10\x9a\x44\x9c\xa0\x11\xe3\x7d\xde\x39\x18\x8e\x29\x3b\x74\xe3\xa3\xa0\xa9\xab\x2e\xab\x3c\xc8\xbd\xcf\x6f\x43\x40\xc0\xb5\xf6\x14\x6e\x5c\xd9\xc6\xad\xfe\x2c\xc2\x12\xff\x87\x0e\x27\x05\x80\x46\xba\x01\xae\xe7\x29\x45\x15\x1f\x56\xb9\x40\x85\x01\xb2\x21\xc0\x28\x4f\xd3\x68\x51\x54\x9c\x0a\xab\x07\x6c\x99\xfb\x23\xca\x07\x85\x3f\x4f\xed\x50\xc7\xcb\xf2\x04\xda\xfb\x9e\xe2\xd0\xd3\xa8\xb3\xf0\x47\xf2\x2f\x12\xc7\xa9\x80\x81\xa6\x08\xf0\x56\xdb\xb2\xb5\xe7\x61\x1e\x4b\x01\x7e\x8f\xce\xd3\x73\x16\xbf\x89\x89\x94\x3e\x75\x52\xde\xda\xf3\x30\xb6\x8a\xb8\xd6\xf0\x94\x51\xc4\x02\x1a\x64\xba\x33\x57\x08\x9e\xf3\x58\x0a\x1c\x04\x4f\xc0\x6d\x3c\x01\xb7\xf1\x04\xdc\xc6\x13\x70\x9b\x23\x2e\x00\xfc\x03\xa8\xc2\xdd\x67\xda\x87\x82\x92\x00\x00\x00\x00\x49\x45\x4e\x44\xae\x42\x60\x82'

image='\x89\x50\x4e\x47\x0d\x0a\x1a\x0a\x00\x00\x00\x0d\x49\x48\x44\x52\x00\x00\x00\x30\x00\x00\x00\x42\x08\x06\x00\x00\x00\x11\xe8\x55\x4d\x00\x00\x00\x01\x73\x52\x47\x42\x00\xae\xce\x1c\xe9\x00\x00\x00\x04\x67\x41\x4d\x41\x00\x00\xb1\x8f\x0b\xfc\x61\x05\x00\x00\x00\x09\x70\x48\x59\x73\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7\x6f\xa8\x64\x00\x00\x04\xd3\x49\x44\x41\x54\x68\x43\xed\x9a\x4d\x4c\xdb\x66\x18\xc7\xff\x8e\xed\x7c\x38\x21\x10\xb2\xad\x1b\x2d\xd0\x42\x37\xc4\x3e\xba\x6a\x55\xa7\x56\x82\x4a\x2b\xea\x26\x55\x48\x54\x3b\x8c\xc3\x24\x34\x4d\xdb\x91\x03\x27\x2e\x1c\xd6\xcb\x2e\x3b\x20\x76\x42\x4c\x3d\xa0\x6a\x07\x90\x36\x36\x69\x07\xb4\x4e\x1b\xe7\x7d\x08\x6d\xad\xd8\x54\x46\x17\xca\x04\x6c\x40\x9c\x90\xc4\x21\x71\xec\x77\xcf\x1b\xac\x55\x5a\x4b\x06\x8e\x83\x8b\xe4\x9f\x64\xe1\xd8\x8e\xfd\xfc\xde\xf7\x79\x1e\x5b\x31\x42\x22\x91\x60\x38\xc2\xf8\xac\xbf\x47\x16\x4f\xe0\x51\xe8\xba\x8e\x85\x85\x05\x18\x86\x61\x6d\xa9\x1d\x35\x13\x98\x9c\x9c\x44\xa1\x50\xb0\xb6\xd4\x0e\xaf\x88\x1f\x45\x26\x93\xc1\xf0\xf0\xf0\xbf\x33\xb0\xb2\xb2\x82\xd1\xd1\x51\xa8\xaa\x8a\xf1\xf1\x71\xf4\xf5\xf5\x61\x6c\x6c\x0c\xb9\x5c\xae\xbc\xbf\x1a\x6a\x56\xc4\x53\x53\x53\xd6\xda\x2e\x3c\xe0\x89\x89\x09\x0c\x0c\x0c\x60\x7a\x7a\x1a\xf9\x7c\x1e\x23\x23\x23\x55\xd7\xc9\xa1\x75\xa1\xde\xde\x5e\x0c\x0d\x0d\x41\x51\x14\x04\x02\x01\x0c\x0e\x0e\x62\x66\x66\x06\xa5\x52\xc9\x3a\xc2\x1e\x87\x26\x60\x9a\x26\x44\x51\xb4\x3e\x01\x92\x24\xa1\xab\xab\xcb\xfa\x64\x9f\x43\x15\xf8\x2f\xbc\x26\xaa\xe5\xd0\x04\x6a\x85\x27\xe0\x36\x35\x13\xe8\xef\xef\xb7\xd6\x76\xe9\xe8\xe8\xb0\xd6\x1e\xd0\xd3\xd3\x63\xad\xd9\xc7\xbb\x13\xbb\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\xd4\x4c\xc0\xa4\x53\xe7\x4d\x19\x29\x5d\x81\xaa\x87\x91\x35\x82\x30\xd8\x83\xdf\x46\x9d\xc2\xf1\x9f\x55\x8a\x4c\xc2\x46\x21\x8c\xd5\x1c\x2d\x9a\x8c\xfb\xdb\xc0\x6f\x1b\xc0\xf7\x9b\x0c\x6a\xd2\xc0\x6b\xc7\x25\x04\x64\xe0\xd2\x69\xe0\xd9\x58\x11\xcf\x28\x39\xc4\xe4\x3c\xfc\xbe\x12\x04\x1c\x3c\x14\x87\x04\x04\x1a\x5d\x1f\x36\x0b\x0a\xe6\x93\x8d\x98\xfd\x1d\xf8\x6a\xd5\x04\xb6\x18\x58\x56\x07\x68\x1f\x24\x01\xf1\xa8\x0f\xc9\x02\x6d\xe7\xaf\x04\xc2\x02\x9e\x8e\x89\xb8\xdc\x2a\xe0\xf5\x93\x06\x3a\xa3\x29\xd4\xfb\xf3\x90\x7c\xb4\x9f\xed\x3f\xa4\xaa\x05\x18\x05\xbf\x63\x4a\x58\xcc\xc6\xf0\xf1\x8f\x21\xcc\xf1\xc0\x69\xa4\x59\xc6\x80\x10\xa5\x94\x89\x4b\xe5\xa3\xb0\x43\x8b\x46\x0b\x05\xcf\xd2\x74\x0c\x09\x81\xbf\xdc\x08\x89\x10\x48\xe4\xcd\x17\x7d\x78\xf7\x79\x0d\xed\x11\x15\xb2\xb0\xff\x77\x06\x55\x09\xf0\xe0\x35\xca\xf3\xf9\xad\xa7\xf0\xce\xac\x0f\x6c\xad\x40\x39\x44\x27\x8d\xca\x78\xe3\x65\x11\x97\x9a\x81\xd6\x68\x89\xd2\x44\x43\x9c\x46\x57\x11\x8b\x48\xea\x21\x24\x8b\x41\xfc\x9a\x8a\xe0\xd6\x12\x30\x7b\x97\x64\x37\x69\x96\xfc\x22\x7c\xed\x32\x6e\x76\x97\x70\xa6\x61\x93\xbc\x8a\xfb\x4a\x29\xfb\x02\x02\x8d\xbc\x21\xe3\x67\xf5\x09\xbc\xfd\x39\x05\xbf\x41\xa3\x29\x99\x10\xda\x82\x18\xbd\x00\xbc\x12\x4f\xe1\xc9\x80\x06\x51\xa0\x72\x16\xb8\x2a\xbf\x0c\x2d\xf4\x3d\xd3\xf4\x95\x67\x6d\xb3\x10\xc2\x2f\xc9\x7a\x8c\xfd\x04\xdc\x5b\x24\x09\xda\x27\x44\x05\xdc\xb8\x26\xe0\xd5\xf8\x3a\x42\x54\x17\xe5\xef\x54\xc0\xa6\x80\x00\x9d\x3a\xca\x72\xae\x1e\xef\x7f\x1b\xc1\xfd\x79\x8d\xd2\xc0\x8f\x73\x9d\x12\xae\x5f\xc8\xa1\x39\x94\x41\x48\xd2\xa9\x0f\x51\xaa\xec\x09\x89\xd0\x52\x32\x45\x2c\xd1\x79\xde\x9b\x0b\x63\xfd\x0e\x49\x98\x3a\x84\x96\x10\xbe\xec\xdb\x41\x47\xdd\x16\x65\x5a\xe5\x77\x68\xb6\xda\x28\xbf\x30\x4f\x83\xcf\xee\x51\xf0\x77\x68\xaa\xeb\xfd\x68\x39\x29\xe2\xc3\x8b\xd9\x72\x0e\x2b\x52\xf1\x7f\x82\xe7\xb0\xf2\x31\xbc\xfb\xb4\x47\x52\xb8\x79\x39\x8b\xf3\xe7\xa8\x3d\xc9\x54\x33\x34\x9b\xa3\x54\x4f\x5b\x45\x85\x8e\xa8\x1c\xa2\x2d\x81\x22\x8d\xda\xdd\x74\x0c\x9f\xfc\x40\x41\x8a\x26\xda\xda\x24\x7c\xd4\x55\x42\x4b\x38\x4d\x05\xc8\xf5\x0e\x32\xa9\x0c\x7e\x1a\xe5\x13\x4a\x1a\x1f\x5c\xd4\x50\x77\x5a\x06\x2b\x9a\x98\x4b\x18\xb8\xad\x36\x96\xaf\x55\x09\x5b\x02\xdb\x7a\x10\xdf\x2c\x53\xde\xff\x5d\x80\x10\x97\x31\x78\x16\x78\xa1\x61\x83\x82\xe7\xd3\x6d\x23\x23\x2d\x89\x66\x92\xb8\x7e\x9e\xea\x80\x5a\x2c\x4b\x9a\xf8\xfa\x0f\x20\x4d\xd7\xe2\x15\xb4\x17\xb6\x04\xd6\xb4\x30\x3e\x5d\xe2\x81\xca\xe8\x6e\x15\x71\x26\x9e\x41\x70\x1f\x05\x57\x19\x46\x9d\x87\x3a\x50\x63\x06\xdd\x9d\x94\x4a\x34\xb9\xdf\xfd\xc9\xb0\xa2\x45\x9c\x17\x58\x4c\x05\x80\x14\xa5\x4a\x88\xe1\xea\x73\xd4\xea\x83\x1a\x6d\xad\x26\xf8\x5d\x78\x4d\x34\x51\x03\xb8\x72\x8a\x4a\x3c\x0c\xa8\xeb\x25\xac\x66\xfc\x30\x99\xc3\x02\xb7\xd7\x28\xdc\xbf\x8a\x74\x97\x05\x5e\x6a\xd4\xa0\xf8\xa8\xf9\x3b\x44\x80\x66\xb2\x25\x62\x42\x8e\x49\x60\x39\x03\x6b\x39\x3e\x34\x0e\x0b\x1c\x8b\x02\x6d\x67\x43\x10\x9a\x44\x9c\xa0\x11\xe3\x7d\xde\x39\x18\x8e\x29\x3b\x74\xe3\xa3\xa0\xa9\xab\x2e\xab\x3c\xc8\xbd\xcf\x6f\x43\x40\xc0\xb5\xf6\x14\x6e\x5c\xd9\xc6\xad\xfe\x2c\xc2\x12\xff\x87\x0e\x27\x05\x80\x46\xba\x01\xae\xe7\x29\x45\x15\x1f\x56\xb9\x40\x85\x01\xb2\x21\xc0\x28\x4f\xd3\x68\x51\x54\x9c\x0a\xab\x07\x6c\x99\xfb\x23\xca\x07\x85\x3f\x4f\xed\x50\xc7\xcb\xf2\x04\xda\xfb\x9e\xe2\xd0\xd3\xa8\xb3\xf0\x47\xf2\x2f\x12\xc7\xa9\x80\x81\xa6\x08\xf0\x56\xdb\xb2\xb5\xe7\x61\x1e\x4b\x01\x7e\x8f\xce\xd3\x73\x16\xbf\x89\x89\x94\x3e\x75\x52\xde\xda\xf3\x30\xb6\x8a\xb8\xd6\xf0\x94\x51\xc4\x02\x1a\x64\xba\x33\x57\x08\x9e\xf3\x58\x0a\x1c\x04\x4f\xc0\x6d\x3c\x01\xb7\xf1\x04\xdc\xc6\x13\x70\x9b\x23\x2e\x00\xfc\x03\xa8\xc2\xdd\x67\xda\x87\x82\x92\x00\x00\x00\x00\x49\x45\x4e\x44\xae\x42\x60\x82'


# Your input data as a dictionary
data = {"search_query":"curtis sittenfeld","image_file":b'\x89\x50\x4e\x47\x0d\x0a\x1a\x0a\x00\x00\x00\x0d\x49\x48\x44\x52\x00\x00\x00\x30\x00\x00\x00\x42\x08\x06\x00\x00\x00\x11\xe8\x55\x4d\x00\x00\x00\x01\x73\x52\x47\x42\x00\xae\xce\x1c\xe9\x00\x00\x00\x04\x67\x41\x4d\x41\x00\x00\xb1\x8f\x0b\xfc\x61\x05\x00\x00\x00\x09\x70\x48\x59\x73\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7\x6f\xa8\x64\x00\x00\x04\xd3\x49\x44\x41\x54\x68\x43\xed\x9a\x4d\x4c\xdb\x66\x18\xc7\xff\x8e\xed\x7c\x38\x21\x10\xb2\xad\x1b\x2d\xd0\x42\x37\xc4\x3e\xba\x6a\x55\xa7\x56\x82\x4a\x2b\xea\x26\x55\x48\x54\x3b\x8c\xc3\x24\x34\x4d\xdb\x91\x03\x27\x2e\x1c\xd6\xcb\x2e\x3b\x20\x76\x42\x4c\x3d\xa0\x6a\x07\x90\x36\x36\x69\x07\xb4\x4e\x1b\xe7\x7d\x08\x6d\xad\xd8\x54\x46\x17\xca\x04\x6c\x40\x9c\x90\xc4\x21\x71\xec\x77\xcf\x1b\xac\x55\x5a\x4b\x06\x8e\x83\x8b\xe4\x9f\x64\xe1\xd8\x8e\xfd\xfc\xde\xf7\x79\x1e\x5b\x31\x42\x22\x91\x60\x38\xc2\xf8\xac\xbf\x47\x16\x4f\xe0\x51\xe8\xba\x8e\x85\x85\x05\x18\x86\x61\x6d\xa9\x1d\x35\x13\x98\x9c\x9c\x44\xa1\x50\xb0\xb6\xd4\x0e\xaf\x88\x1f\x45\x26\x93\xc1\xf0\xf0\xf0\xbf\x33\xb0\xb2\xb2\x82\xd1\xd1\x51\xa8\xaa\x8a\xf1\xf1\x71\xf4\xf5\xf5\x61\x6c\x6c\x0c\xb9\x5c\xae\xbc\xbf\x1a\x6a\x56\xc4\x53\x53\x53\xd6\xda\x2e\x3c\xe0\x89\x89\x09\x0c\x0c\x0c\x60\x7a\x7a\x1a\xf9\x7c\x1e\x23\x23\x23\x55\xd7\xc9\xa1\x75\xa1\xde\xde\x5e\x0c\x0d\x0d\x41\x51\x14\x04\x02\x01\x0c\x0e\x0e\x62\x66\x66\x06\xa5\x52\xc9\x3a\xc2\x1e\x87\x26\x60\x9a\x26\x44\x51\xb4\x3e\x01\x92\x24\xa1\xab\xab\xcb\xfa\x64\x9f\x43\x15\xf8\x2f\xbc\x26\xaa\xe5\xd0\x04\x6a\x85\x27\xe0\x36\x35\x13\xe8\xef\xef\xb7\xd6\x76\xe9\xe8\xe8\xb0\xd6\x1e\xd0\xd3\xd3\x63\xad\xd9\xc7\xbb\x13\xbb\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\xd4\x4c\xc0\xa4\x53\xe7\x4d\x19\x29\x5d\x81\xaa\x87\x91\x35\x82\x30\xd8\x83\xdf\x46\x9d\xc2\xf1\x9f\x55\x8a\x4c\xc2\x46\x21\x8c\xd5\x1c\x2d\x9a\x8c\xfb\xdb\xc0\x6f\x1b\xc0\xf7\x9b\x0c\x6a\xd2\xc0\x6b\xc7\x25\x04\x64\xe0\xd2\x69\xe0\xd9\x58\x11\xcf\x28\x39\xc4\xe4\x3c\xfc\xbe\x12\x04\x1c\x3c\x14\x87\x04\x04\x1a\x5d\x1f\x36\x0b\x0a\xe6\x93\x8d\x98\xfd\x1d\xf8\x6a\xd5\x04\xb6\x18\x58\x56\x07\x68\x1f\x24\x01\xf1\xa8\x0f\xc9\x02\x6d\xe7\xaf\x04\xc2\x02\x9e\x8e\x89\xb8\xdc\x2a\xe0\xf5\x93\x06\x3a\xa3\x29\xd4\xfb\xf3\x90\x7c\xb4\x9f\xed\x3f\xa4\xaa\x05\x18\x05\xbf\x63\x4a\x58\xcc\xc6\xf0\xf1\x8f\x21\xcc\xf1\xc0\x69\xa4\x59\xc6\x80\x10\xa5\x94\x89\x4b\xe5\xa3\xb0\x43\x8b\x46\x0b\x05\xcf\xd2\x74\x0c\x09\x81\xbf\xdc\x08\x89\x10\x48\xe4\xcd\x17\x7d\x78\xf7\x79\x0d\xed\x11\x15\xb2\xb0\xff\x77\x06\x55\x09\xf0\xe0\x35\xca\xf3\xf9\xad\xa7\xf0\xce\xac\x0f\x6c\xad\x40\x39\x44\x27\x8d\xca\x78\xe3\x65\x11\x97\x9a\x81\xd6\x68\x89\xd2\x44\x43\x9c\x46\x57\x11\x8b\x48\xea\x21\x24\x8b\x41\xfc\x9a\x8a\xe0\xd6\x12\x30\x7b\x97\x64\x37\x69\x96\xfc\x22\x7c\xed\x32\x6e\x76\x97\x70\xa6\x61\x93\xbc\x8a\xfb\x4a\x29\xfb\x02\x02\x8d\xbc\x21\xe3\x67\xf5\x09\xbc\xfd\x39\x05\xbf\x41\xa3\x29\x99\x10\xda\x82\x18\xbd\x00\xbc\x12\x4f\xe1\xc9\x80\x06\x51\xa0\x72\x16\xb8\x2a\xbf\x0c\x2d\xf4\x3d\xd3\xf4\x95\x67\x6d\xb3\x10\xc2\x2f\xc9\x7a\x8c\xfd\x04\xdc\x5b\x24\x09\xda\x27\x44\x05\xdc\xb8\x26\xe0\xd5\xf8\x3a\x42\x54\x17\xe5\xef\x54\xc0\xa6\x80\x00\x9d\x3a\xca\x72\xae\x1e\xef\x7f\x1b\xc1\xfd\x79\x8d\xd2\xc0\x8f\x73\x9d\x12\xae\x5f\xc8\xa1\x39\x94\x41\x48\xd2\xa9\x0f\x51\xaa\xec\x09\x89\xd0\x52\x32\x45\x2c\xd1\x79\xde\x9b\x0b\x63\xfd\x0e\x49\x98\x3a\x84\x96\x10\xbe\xec\xdb\x41\x47\xdd\x16\x65\x5a\xe5\x77\x68\xb6\xda\x28\xbf\x30\x4f\x83\xcf\xee\x51\xf0\x77\x68\xaa\xeb\xfd\x68\x39\x29\xe2\xc3\x8b\xd9\x72\x0e\x2b\x52\xf1\x7f\x82\xe7\xb0\xf2\x31\xbc\xfb\xb4\x47\x52\xb8\x79\x39\x8b\xf3\xe7\xa8\x3d\xc9\x54\x33\x34\x9b\xa3\x54\x4f\x5b\x45\x85\x8e\xa8\x1c\xa2\x2d\x81\x22\x8d\xda\xdd\x74\x0c\x9f\xfc\x40\x41\x8a\x26\xda\xda\x24\x7c\xd4\x55\x42\x4b\x38\x4d\x05\xc8\xf5\x0e\x32\xa9\x0c\x7e\x1a\xe5\x13\x4a\x1a\x1f\x5c\xd4\x50\x77\x5a\x06\x2b\x9a\x98\x4b\x18\xb8\xad\x36\x96\xaf\x55\x09\x5b\x02\xdb\x7a\x10\xdf\x2c\x53\xde\xff\x5d\x80\x10\x97\x31\x78\x16\x78\xa1\x61\x83\x82\xe7\xd3\x6d\x23\x23\x2d\x89\x66\x92\xb8\x7e\x9e\xea\x80\x5a\x2c\x4b\x9a\xf8\xfa\x0f\x20\x4d\xd7\xe2\x15\xb4\x17\xb6\x04\xd6\xb4\x30\x3e\x5d\xe2\x81\xca\xe8\x6e\x15\x71\x26\x9e\x41\x70\x1f\x05\x57\x19\x46\x9d\x87\x3a\x50\x63\x06\xdd\x9d\x94\x4a\x34\xb9\xdf\xfd\xc9\xb0\xa2\x45\x9c\x17\x58\x4c\x05\x80\x14\xa5\x4a\x88\xe1\xea\x73\xd4\xea\x83\x1a\x6d\xad\x26\xf8\x5d\x78\x4d\x34\x51\x03\xb8\x72\x8a\x4a\x3c\x0c\xa8\xeb\x25\xac\x66\xfc\x30\x99\xc3\x02\xb7\xd7\x28\xdc\xbf\x8a\x74\x97\x05\x5e\x6a\xd4\xa0\xf8\xa8\xf9\x3b\x44\x80\x66\xb2\x25\x62\x42\x8e\x49\x60\x39\x03\x6b\x39\x3e\x34\x0e\x0b\x1c\x8b\x02\x6d\x67\x43\x10\x9a\x44\x9c\xa0\x11\xe3\x7d\xde\x39\x18\x8e\x29\x3b\x74\xe3\xa3\xa0\xa9\xab\x2e\xab\x3c\xc8\xbd\xcf\x6f\x43\x40\xc0\xb5\xf6\x14\x6e\x5c\xd9\xc6\xad\xfe\x2c\xc2\x12\xff\x87\x0e\x27\x05\x80\x46\xba\x01\xae\xe7\x29\x45\x15\x1f\x56\xb9\x40\x85\x01\xb2\x21\xc0\x28\x4f\xd3\x68\x51\x54\x9c\x0a\xab\x07\x6c\x99\xfb\x23\xca\x07\x85\x3f\x4f\xed\x50\xc7\xcb\xf2\x04\xda\xfb\x9e\xe2\xd0\xd3\xa8\xb3\xf0\x47\xf2\x2f\x12\xc7\xa9\x80\x81\xa6\x08\xf0\x56\xdb\xb2\xb5\xe7\x61\x1e\x4b\x01\x7e\x8f\xce\xd3\x73\x16\xbf\x89\x89\x94\x3e\x75\x52\xde\xda\xf3\x30\xb6\x8a\xb8\xd6\xf0\x94\x51\xc4\x02\x1a\x64\xba\x33\x57\x08\x9e\xf3\x58\x0a\x1c\x04\x4f\xc0\x6d\x3c\x01\xb7\xf1\x04\xdc\xc6\x13\x70\x9b\x23\x2e\x00\xfc\x03\xa8\xc2\xdd\x67\xda\x87\x82\x92\x00\x00\x00\x00\x49\x45\x4e\x44\xae\x42\x60\x82'
,"project":"nine-quality-test","region":"us-central1"}

# Simulating an HTTP request with the mock object
mock_request = Mock()
mock_request.get_json.return_value = data  # Mock the get_json method to return your data


In [89]:
image=b'\x89\x50\x4e\x47\x0d\x0a\x1a\x0a\x00\x00\x00\x0d\x49\x48\x44\x52\x00\x00\x00\x30\x00\x00\x00\x42\x08\x06\x00\x00\x00\x11\xe8\x55\x4d\x00\x00\x00\x01\x73\x52\x47\x42\x00\xae\xce\x1c\xe9\x00\x00\x00\x04\x67\x41\x4d\x41\x00\x00\xb1\x8f\x0b\xfc\x61\x05\x00\x00\x00\x09\x70\x48\x59\x73\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7\x6f\xa8\x64\x00\x00\x04\xd3\x49\x44\x41\x54\x68\x43\xed\x9a\x4d\x4c\xdb\x66\x18\xc7\xff\x8e\xed\x7c\x38\x21\x10\xb2\xad\x1b\x2d\xd0\x42\x37\xc4\x3e\xba\x6a\x55\xa7\x56\x82\x4a\x2b\xea\x26\x55\x48\x54\x3b\x8c\xc3\x24\x34\x4d\xdb\x91\x03\x27\x2e\x1c\xd6\xcb\x2e\x3b\x20\x76\x42\x4c\x3d\xa0\x6a\x07\x90\x36\x36\x69\x07\xb4\x4e\x1b\xe7\x7d\x08\x6d\xad\xd8\x54\x46\x17\xca\x04\x6c\x40\x9c\x90\xc4\x21\x71\xec\x77\xcf\x1b\xac\x55\x5a\x4b\x06\x8e\x83\x8b\xe4\x9f\x64\xe1\xd8\x8e\xfd\xfc\xde\xf7\x79\x1e\x5b\x31\x42\x22\x91\x60\x38\xc2\xf8\xac\xbf\x47\x16\x4f\xe0\x51\xe8\xba\x8e\x85\x85\x05\x18\x86\x61\x6d\xa9\x1d\x35\x13\x98\x9c\x9c\x44\xa1\x50\xb0\xb6\xd4\x0e\xaf\x88\x1f\x45\x26\x93\xc1\xf0\xf0\xf0\xbf\x33\xb0\xb2\xb2\x82\xd1\xd1\x51\xa8\xaa\x8a\xf1\xf1\x71\xf4\xf5\xf5\x61\x6c\x6c\x0c\xb9\x5c\xae\xbc\xbf\x1a\x6a\x56\xc4\x53\x53\x53\xd6\xda\x2e\x3c\xe0\x89\x89\x09\x0c\x0c\x0c\x60\x7a\x7a\x1a\xf9\x7c\x1e\x23\x23\x23\x55\xd7\xc9\xa1\x75\xa1\xde\xde\x5e\x0c\x0d\x0d\x41\x51\x14\x04\x02\x01\x0c\x0e\x0e\x62\x66\x66\x06\xa5\x52\xc9\x3a\xc2\x1e\x87\x26\x60\x9a\x26\x44\x51\xb4\x3e\x01\x92\x24\xa1\xab\xab\xcb\xfa\x64\x9f\x43\x15\xf8\x2f\xbc\x26\xaa\xe5\xd0\x04\x6a\x85\x27\xe0\x36\x35\x13\xe8\xef\xef\xb7\xd6\x76\xe9\xe8\xe8\xb0\xd6\x1e\xd0\xd3\xd3\x63\xad\xd9\xc7\xbb\x13\xbb\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\xd4\x4c\xc0\xa4\x53\xe7\x4d\x19\x29\x5d\x81\xaa\x87\x91\x35\x82\x30\xd8\x83\xdf\x46\x9d\xc2\xf1\x9f\x55\x8a\x4c\xc2\x46\x21\x8c\xd5\x1c\x2d\x9a\x8c\xfb\xdb\xc0\x6f\x1b\xc0\xf7\x9b\x0c\x6a\xd2\xc0\x6b\xc7\x25\x04\x64\xe0\xd2\x69\xe0\xd9\x58\x11\xcf\x28\x39\xc4\xe4\x3c\xfc\xbe\x12\x04\x1c\x3c\x14\x87\x04\x04\x1a\x5d\x1f\x36\x0b\x0a\xe6\x93\x8d\x98\xfd\x1d\xf8\x6a\xd5\x04\xb6\x18\x58\x56\x07\x68\x1f\x24\x01\xf1\xa8\x0f\xc9\x02\x6d\xe7\xaf\x04\xc2\x02\x9e\x8e\x89\xb8\xdc\x2a\xe0\xf5\x93\x06\x3a\xa3\x29\xd4\xfb\xf3\x90\x7c\xb4\x9f\xed\x3f\xa4\xaa\x05\x18\x05\xbf\x63\x4a\x58\xcc\xc6\xf0\xf1\x8f\x21\xcc\xf1\xc0\x69\xa4\x59\xc6\x80\x10\xa5\x94\x89\x4b\xe5\xa3\xb0\x43\x8b\x46\x0b\x05\xcf\xd2\x74\x0c\x09\x81\xbf\xdc\x08\x89\x10\x48\xe4\xcd\x17\x7d\x78\xf7\x79\x0d\xed\x11\x15\xb2\xb0\xff\x77\x06\x55\x09\xf0\xe0\x35\xca\xf3\xf9\xad\xa7\xf0\xce\xac\x0f\x6c\xad\x40\x39\x44\x27\x8d\xca\x78\xe3\x65\x11\x97\x9a\x81\xd6\x68\x89\xd2\x44\x43\x9c\x46\x57\x11\x8b\x48\xea\x21\x24\x8b\x41\xfc\x9a\x8a\xe0\xd6\x12\x30\x7b\x97\x64\x37\x69\x96\xfc\x22\x7c\xed\x32\x6e\x76\x97\x70\xa6\x61\x93\xbc\x8a\xfb\x4a\x29\xfb\x02\x02\x8d\xbc\x21\xe3\x67\xf5\x09\xbc\xfd\x39\x05\xbf\x41\xa3\x29\x99\x10\xda\x82\x18\xbd\x00\xbc\x12\x4f\xe1\xc9\x80\x06\x51\xa0\x72\x16\xb8\x2a\xbf\x0c\x2d\xf4\x3d\xd3\xf4\x95\x67\x6d\xb3\x10\xc2\x2f\xc9\x7a\x8c\xfd\x04\xdc\x5b\x24\x09\xda\x27\x44\x05\xdc\xb8\x26\xe0\xd5\xf8\x3a\x42\x54\x17\xe5\xef\x54\xc0\xa6\x80\x00\x9d\x3a\xca\x72\xae\x1e\xef\x7f\x1b\xc1\xfd\x79\x8d\xd2\xc0\x8f\x73\x9d\x12\xae\x5f\xc8\xa1\x39\x94\x41\x48\xd2\xa9\x0f\x51\xaa\xec\x09\x89\xd0\x52\x32\x45\x2c\xd1\x79\xde\x9b\x0b\x63\xfd\x0e\x49\x98\x3a\x84\x96\x10\xbe\xec\xdb\x41\x47\xdd\x16\x65\x5a\xe5\x77\x68\xb6\xda\x28\xbf\x30\x4f\x83\xcf\xee\x51\xf0\x77\x68\xaa\xeb\xfd\x68\x39\x29\xe2\xc3\x8b\xd9\x72\x0e\x2b\x52\xf1\x7f\x82\xe7\xb0\xf2\x31\xbc\xfb\xb4\x47\x52\xb8\x79\x39\x8b\xf3\xe7\xa8\x3d\xc9\x54\x33\x34\x9b\xa3\x54\x4f\x5b\x45\x85\x8e\xa8\x1c\xa2\x2d\x81\x22\x8d\xda\xdd\x74\x0c\x9f\xfc\x40\x41\x8a\x26\xda\xda\x24\x7c\xd4\x55\x42\x4b\x38\x4d\x05\xc8\xf5\x0e\x32\xa9\x0c\x7e\x1a\xe5\x13\x4a\x1a\x1f\x5c\xd4\x50\x77\x5a\x06\x2b\x9a\x98\x4b\x18\xb8\xad\x36\x96\xaf\x55\x09\x5b\x02\xdb\x7a\x10\xdf\x2c\x53\xde\xff\x5d\x80\x10\x97\x31\x78\x16\x78\xa1\x61\x83\x82\xe7\xd3\x6d\x23\x23\x2d\x89\x66\x92\xb8\x7e\x9e\xea\x80\x5a\x2c\x4b\x9a\xf8\xfa\x0f\x20\x4d\xd7\xe2\x15\xb4\x17\xb6\x04\xd6\xb4\x30\x3e\x5d\xe2\x81\xca\xe8\x6e\x15\x71\x26\x9e\x41\x70\x1f\x05\x57\x19\x46\x9d\x87\x3a\x50\x63\x06\xdd\x9d\x94\x4a\x34\xb9\xdf\xfd\xc9\xb0\xa2\x45\x9c\x17\x58\x4c\x05\x80\x14\xa5\x4a\x88\xe1\xea\x73\xd4\xea\x83\x1a\x6d\xad\x26\xf8\x5d\x78\x4d\x34\x51\x03\xb8\x72\x8a\x4a\x3c\x0c\xa8\xeb\x25\xac\x66\xfc\x30\x99\xc3\x02\xb7\xd7\x28\xdc\xbf\x8a\x74\x97\x05\x5e\x6a\xd4\xa0\xf8\xa8\xf9\x3b\x44\x80\x66\xb2\x25\x62\x42\x8e\x49\x60\x39\x03\x6b\x39\x3e\x34\x0e\x0b\x1c\x8b\x02\x6d\x67\x43\x10\x9a\x44\x9c\xa0\x11\xe3\x7d\xde\x39\x18\x8e\x29\x3b\x74\xe3\xa3\xa0\xa9\xab\x2e\xab\x3c\xc8\xbd\xcf\x6f\x43\x40\xc0\xb5\xf6\x14\x6e\x5c\xd9\xc6\xad\xfe\x2c\xc2\x12\xff\x87\x0e\x27\x05\x80\x46\xba\x01\xae\xe7\x29\x45\x15\x1f\x56\xb9\x40\x85\x01\xb2\x21\xc0\x28\x4f\xd3\x68\x51\x54\x9c\x0a\xab\x07\x6c\x99\xfb\x23\xca\x07\x85\x3f\x4f\xed\x50\xc7\xcb\xf2\x04\xda\xfb\x9e\xe2\xd0\xd3\xa8\xb3\xf0\x47\xf2\x2f\x12\xc7\xa9\x80\x81\xa6\x08\xf0\x56\xdb\xb2\xb5\xe7\x61\x1e\x4b\x01\x7e\x8f\xce\xd3\x73\x16\xbf\x89\x89\x94\x3e\x75\x52\xde\xda\xf3\x30\xb6\x8a\xb8\xd6\xf0\x94\x51\xc4\x02\x1a\x64\xba\x33\x57\x08\x9e\xf3\x58\x0a\x1c\x04\x4f\xc0\x6d\x3c\x01\xb7\xf1\x04\xdc\xc6\x13\x70\x9b\x23\x2e\x00\xfc\x03\xa8\xc2\xdd\x67\xda\x87\x82\x92\x00\x00\x00\x00\x49\x45\x4e\x44\xae\x42\x60\x82'

# Encode the image bytes to Base64
encoded_image = base64.b64encode(image_byte_string).decode('utf-8')
encoded_image

'/9j/4QDKRXhpZgAATU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAABAAAAXgEoAAMAAAABAAIAAAITAAMAAAABAAEAAIdpAAQAAAABAAAAZgAAAAAAAAEsAAAAAQAAASwAAAABAAeQAAAHAAAABDAyMjGRAQAHAAAABAECAwCgAAAHAAAABDAxMDCgAQADAAAAAQABAACgAgAEAAAAAQAAAoCgAwAEAAAAAQAAAaukBgADAAAAAQAAAAAAAAAAAAD/4gJASUNDX1BST0ZJTEUAAQEAAAIwQURCRQIQAABtbnRyUkdCIFhZWiAH0AAIAAsAEwAzADthY3NwQVBQTAAAAABub25lAAAAAAAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUFEQkUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAApjcHJ0AAAA/AAAADJkZXNjAAABMAAAAGt3dHB0AAABnAAAABRia3B0AAABsAAAABRyVFJDAAABxAAAAA5nVFJDAAAB1AAAAA5iVFJDAAAB5AAAAA5yWFlaAAAB9AAAABRnWFlaAAACCAAAABRiWFlaAAACHAAAABR0ZXh0AAAAAENvcHlyaWdodCAyMDAwIEFkb2JlIFN5c3RlbXMgSW5jb3Jwb3JhdGVkAAAAZGVzYwAAAAAAAAARQWRvYmUgUkdCICgxOTk4KQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAGN1cnYAAAAAAAAAAQIzAABjdXJ2AAAAAAAAAAECMwAAY3VydgAAAAAAAAABAjMAAFhZWiAAAAAAAACcGAAAT6UAAAT8WFlaIAA

In [69]:
img='iVBORw0KGgoAAAANSUhEUgAAADAAAABCCAYAAAAR6FVNAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAATTSURBVGhD7ZpNTNtmGMf/ju18OCEQsq0bLdBCN8Q+umpVp1aCSivqJlVIVDuMwyQ0TduRAycuHNbLLjsgdkJMPaBqB5A2NmkHtE4b530Iba3YVEYXygRsQJyQxCFx7HfPG6xVWksGjoOL5J9k4diO/fze93keWzFCIpFgOML4rL9HFk/gUei6joWFBRiGYW2pHTUTmJycRKFQsLbUDq+IH0Umk8Hw8PC/M7CysoLR0VGoqorx8XH09fVhbGwMuVyuvL8aalbEU1NT1touPOCJiQkMDAxgenoa+XweIyMjVdfJoXWh3t5eDA0NQVEUBAIBDA4OYmZmBqVSyTrCHocmYJomRFG0PgGSJKGrq8v6ZJ9DFfgvvCaq5dAEaoUn4DY1E+jv77fWduno6LDWHtDT02Ot2ce7E7uNJ+A2noDbeAJu4wm4jSfgNp6A23gCbuMJuI0n4DaegNvUTMCkU+dNGSldgaqHkTWCMNiD30adwvGfVYpMwkYhjNUcLZqM+9vAbxvA95sMatLAa8clBGTg0mng2VgRzyg5xOQ8/L4SBBw8FIcEBBpdHzYLCuaTjZj9Hfhq1QS2GFhWB2gfJAHxqA/JAm3nrwTCAp6OibjcKuD1kwY6oynU+/OQfLSf7T+kqgUYBb9jSljMxvDxjyHM8cBppFnGgBCllIlL5aOwQ4tGCwXP0nQMCYG/3AiJEEjkzRd9ePd5De0RFbKw/3cGVQnw4DXK8/mtp/DOrA9srUA5RCeNynjjZRGXmoHWaInSREOcRlcRi0jqISSLQfyaiuDWEjB7l2Q3aZb8InztMm52l3CmYZO8ivtKKfsCAo28IeNn9Qm8/TkFv0GjKZkQ2oIYvQC8Ek/hyYAGUaByFrgqvwwt9D3T9JVnbbMQwi/Jeoz9BNxbJAnaJ0QF3Lgm4NX4OkJUF+XvVMCmgACdOspyrh7vfxvB/XmN0sCPc50Srl/IoTmUQUjSqQ9RquwJidBSMkUs0Xnemwtj/Q5JmDqElhC+7NtBR90WZVrld2i22ii/ME+Dz+5R8Hdoquv9aDkp4sOL2XIOK1Lxf4LnsPIxvPu0R1K4eTmL8+eoPclUMzSbo1RPW0WFjqgcoi2BIo3a3XQMn/xAQYom2tokfNRVQks4TQXI9Q4yqQx+GuUTShofXNRQd1oGK5qYSxi4rTaWr1UJWwLbehDfLFPe/12AEJcxeBZ4oWGDgufTbSMjLYlmkrh+nuqAWixLmvj6DyBN1+IVtBe2BNa0MD5d4oHK6G4VcSaeQXAfBVcZRp2HOlBjBt2dlEo0ud/9ybCiRZwXWEwFgBSlSojh6nPU6oMaba0m+F14TTRRA7hyiko8DKjrJaxm/DCZwwK31yjcv4p0lwVeatSg+Kj5O0SAZrIlYkKOSWA5A2s5PjQOCxyLAm1nQxCaRJygEeN93jkYjik7dOOjoKmrLqs8yL3Pb0NAwLX2FG5c2cat/izCEv+HDicFgEa6Aa7nKUUVH1a5QIUBsiHAKE/TaFFUnAqrB2yZ+yPKB4U/T+1Qx8vyBNr7nuLQ06iz8EfyLxLHqYCBpgjwVtuytedhHksBfo/O03MWv4mJlD51Ut7a8zC2irjW8JRRxAIaZLozVwie81gKHARPwG08AbfxBNzGE3CbIy4A/AOowt1n2oeCkgAAAABJRU5ErkJggg=='

In [90]:
image_bytes = base64.b64decode(img)


In [62]:
import base64
i= "\x89\x50\x4e\x47\x0d\x0a\x1a\x0a\x00\x00\x00\x0d\x49\x48\x44\x52\x00\x00\x00\x30\x00\x00\x00\x42\x08\x06\x00\x00\x00\x11\xe8\x55\x4d\x00\x00\x00\x01\x73\x52\x47\x42\x00\xae\xce\x1c\xe9\x00\x00\x00\x04\x67\x41\x4d\x41\x00\x00\xb1\x8f\x0b\xfc\x61\x05\x00\x00\x00\x09\x70\x48\x59\x73\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7\x6f\xa8\x64\x00\x00\x04\xd3\x49\x44\x41\x54\x68\x43\xed\x9a\x4d\x4c\xdb\x66\x18\xc7\xff\x8e\xed\x7c\x38\x21\x10\xb2\xad\x1b\x2d\xd0\x42\x37\xc4\x3e\xba\x6a\x55\xa7\x56\x82\x4a\x2b\xea\x26\x55\x48\x54\x3b\x8c\xc3\x24\x34\x4d\xdb\x91\x03\x27\x2e\x1c\xd6\xcb\x2e\x3b\x20\x76\x42\x4c\x3d\xa0\x6a\x07\x90\x36\x36\x69\x07\xb4\x4e\x1b\xe7\x7d\x08\x6d\xad\xd8\x54\x46\x17\xca\x04\x6c\x40\x9c\x90\xc4\x21\x71\xec\x77\xcf\x1b\xac\x55\x5a\x4b\x06\x8e\x83\x8b\xe4\x9f\x64\xe1\xd8\x8e\xfd\xfc\xde\xf7\x79\x1e\x5b\x31\x42\x22\x91\x60\x38\xc2\xf8\xac\xbf\x47\x16\x4f\xe0\x51\xe8\xba\x8e\x85\x85\x05\x18\x86\x61\x6d\xa9\x1d\x35\x13\x98\x9c\x9c\x44\xa1\x50\xb0\xb6\xd4\x0e\xaf\x88\x1f\x45\x26\x93\xc1\xf0\xf0\xf0\xbf\x33\xb0\xb2\xb2\x82\xd1\xd1\x51\xa8\xaa\x8a\xf1\xf1\x71\xf4\xf5\xf5\x61\x6c\x6c\x0c\xb9\x5c\xae\xbc\xbf\x1a\x6a\x56\xc4\x53\x53\x53\xd6\xda\x2e\x3c\xe0\x89\x89\x09\x0c\x0c\x0c\x60\x7a\x7a\x1a\xf9\x7c\x1e\x23\x23\x23\x55\xd7\xc9\xa1\x75\xa1\xde\xde\x5e\x0c\x0d\x0d\x41\x51\x14\x04\x02\x01\x0c\x0e\x0e\x62\x66\x66\x06\xa5\x52\xc9\x3a\xc2\x1e\x87\x26\x60\x9a\x26\x44\x51\xb4\x3e\x01\x92\x24\xa1\xab\xab\xcb\xfa\x64\x9f\x43\x15\xf8\x2f\xbc\x26\xaa\xe5\xd0\x04\x6a\x85\x27\xe0\x36\x35\x13\xe8\xef\xef\xb7\xd6\x76\xe9\xe8\xe8\xb0\xd6\x1e\xd0\xd3\xd3\x63\xad\xd9\xc7\xbb\x13\xbb\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\x78\x02\x6e\xe3\x09\xb8\x8d\x27\xe0\x36\x9e\x80\xdb\xd4\x4c\xc0\xa4\x53\xe7\x4d\x19\x29\x5d\x81\xaa\x87\x91\x35\x82\x30\xd8\x83\xdf\x46\x9d\xc2\xf1\x9f\x55\x8a\x4c\xc2\x46\x21\x8c\xd5\x1c\x2d\x9a\x8c\xfb\xdb\xc0\x6f\x1b\xc0\xf7\x9b\x0c\x6a\xd2\xc0\x6b\xc7\x25\x04\x64\xe0\xd2\x69\xe0\xd9\x58\x11\xcf\x28\x39\xc4\xe4\x3c\xfc\xbe\x12\x04\x1c\x3c\x14\x87\x04\x04\x1a\x5d\x1f\x36\x0b\x0a\xe6\x93\x8d\x98\xfd\x1d\xf8\x6a\xd5\x04\xb6\x18\x58\x56\x07\x68\x1f\x24\x01\xf1\xa8\x0f\xc9\x02\x6d\xe7\xaf\x04\xc2\x02\x9e\x8e\x89\xb8\xdc\x2a\xe0\xf5\x93\x06\x3a\xa3\x29\xd4\xfb\xf3\x90\x7c\xb4\x9f\xed\x3f\xa4\xaa\x05\x18\x05\xbf\x63\x4a\x58\xcc\xc6\xf0\xf1\x8f\x21\xcc\xf1\xc0\x69\xa4\x59\xc6\x80\x10\xa5\x94\x89\x4b\xe5\xa3\xb0\x43\x8b\x46\x0b\x05\xcf\xd2\x74\x0c\x09\x81\xbf\xdc\x08\x89\x10\x48\xe4\xcd\x17\x7d\x78\xf7\x79\x0d\xed\x11\x15\xb2\xb0\xff\x77\x06\x55\x09\xf0\xe0\x35\xca\xf3\xf9\xad\xa7\xf0\xce\xac\x0f\x6c\xad\x40\x39\x44\x27\x8d\xca\x78\xe3\x65\x11\x97\x9a\x81\xd6\x68\x89\xd2\x44\x43\x9c\x46\x57\x11\x8b\x48\xea\x21\x24\x8b\x41\xfc\x9a\x8a\xe0\xd6\x12\x30\x7b\x97\x64\x37\x69\x96\xfc\x22\x7c\xed\x32\x6e\x76\x97\x70\xa6\x61\x93\xbc\x8a\xfb\x4a\x29\xfb\x02\x02\x8d\xbc\x21\xe3\x67\xf5\x09\xbc\xfd\x39\x05\xbf\x41\xa3\x29\x99\x10\xda\x82\x18\xbd\x00\xbc\x12\x4f\xe1\xc9\x80\x06\x51\xa0\x72\x16\xb8\x2a\xbf\x0c\x2d\xf4\x3d\xd3\xf4\x95\x67\x6d\xb3\x10\xc2\x2f\xc9\x7a\x8c\xfd\x04\xdc\x5b\x24\x09\xda\x27\x44\x05\xdc\xb8\x26\xe0\xd5\xf8\x3a\x42\x54\x17\xe5\xef\x54\xc0\xa6\x80\x00\x9d\x3a\xca\x72\xae\x1e\xef\x7f\x1b\xc1\xfd\x79\x8d\xd2\xc0\x8f\x73\x9d\x12\xae\x5f\xc8\xa1\x39\x94\x41\x48\xd2\xa9\x0f\x51\xaa\xec\x09\x89\xd0\x52\x32\x45\x2c\xd1\x79\xde\x9b\x0b\x63\xfd\x0e\x49\x98\x3a\x84\x96\x10\xbe\xec\xdb\x41\x47\xdd\x16\x65\x5a\xe5\x77\x68\xb6\xda\x28\xbf\x30\x4f\x83\xcf\xee\x51\xf0\x77\x68\xaa\xeb\xfd\x68\x39\x29\xe2\xc3\x8b\xd9\x72\x0e\x2b\x52\xf1\x7f\x82\xe7\xb0\xf2\x31\xbc\xfb\xb4\x47\x52\xb8\x79\x39\x8b\xf3\xe7\xa8\x3d\xc9\x54\x33\x34\x9b\xa3\x54\x4f\x5b\x45\x85\x8e\xa8\x1c\xa2\x2d\x81\x22\x8d\xda\xdd\x74\x0c\x9f\xfc\x40\x41\x8a\x26\xda\xda\x24\x7c\xd4\x55\x42\x4b\x38\x4d\x05\xc8\xf5\x0e\x32\xa9\x0c\x7e\x1a\xe5\x13\x4a\x1a\x1f\x5c\xd4\x50\x77\x5a\x06\x2b\x9a\x98\x4b\x18\xb8\xad\x36\x96\xaf\x55\x09\x5b\x02\xdb\x7a\x10\xdf\x2c\x53\xde\xff\x5d\x80\x10\x97\x31\x78\x16\x78\xa1\x61\x83\x82\xe7\xd3\x6d\x23\x23\x2d\x89\x66\x92\xb8\x7e\x9e\xea\x80\x5a\x2c\x4b\x9a\xf8\xfa\x0f\x20\x4d\xd7\xe2\x15\xb4\x17\xb6\x04\xd6\xb4\x30\x3e\x5d\xe2\x81\xca\xe8\x6e\x15\x71\x26\x9e\x41\x70\x1f\x05\x57\x19\x46\x9d\x87\x3a\x50\x63\x06\xdd\x9d\x94\x4a\x34\xb9\xdf\xfd\xc9\xb0\xa2\x45\x9c\x17\x58\x4c\x05\x80\x14\xa5\x4a\x88\xe1\xea\x73\xd4\xea\x83\x1a\x6d\xad\x26\xf8\x5d\x78\x4d\x34\x51\x03\xb8\x72\x8a\x4a\x3c\x0c\xa8\xeb\x25\xac\x66\xfc\x30\x99\xc3\x02\xb7\xd7\x28\xdc\xbf\x8a\x74\x97\x05\x5e\x6a\xd4\xa0\xf8\xa8\xf9\x3b\x44\x80\x66\xb2\x25\x62\x42\x8e\x49\x60\x39\x03\x6b\x39\x3e\x34\x0e\x0b\x1c\x8b\x02\x6d\x67\x43\x10\x9a\x44\x9c\xa0\x11\xe3\x7d\xde\x39\x18\x8e\x29\x3b\x74\xe3\xa3\xa0\xa9\xab\x2e\xab\x3c\xc8\xbd\xcf\x6f\x43\x40\xc0\xb5\xf6\x14\x6e\x5c\xd9\xc6\xad\xfe\x2c\xc2\x12\xff\x87\x0e\x27\x05\x80\x46\xba\x01\xae\xe7\x29\x45\x15\x1f\x56\xb9\x40\x85\x01\xb2\x21\xc0\x28\x4f\xd3\x68\x51\x54\x9c\x0a\xab\x07\x6c\x99\xfb\x23\xca\x07\x85\x3f\x4f\xed\x50\xc7\xcb\xf2\x04\xda\xfb\x9e\xe2\xd0\xd3\xa8\xb3\xf0\x47\xf2\x2f\x12\xc7\xa9\x80\x81\xa6\x08\xf0\x56\xdb\xb2\xb5\xe7\x61\x1e\x4b\x01\x7e\x8f\xce\xd3\x73\x16\xbf\x89\x89\x94\x3e\x75\x52\xde\xda\xf3\x30\xb6\x8a\xb8\xd6\xf0\x94\x51\xc4\x02\x1a\x64\xba\x33\x57\x08\x9e\xf3\x58\x0a\x1c\x04\x4f\xc0\x6d\x3c\x01\xb7\xf1\x04\xdc\xc6\x13\x70\x9b\x23\x2e\x00\xfc\x03\xa8\xc2\xdd\x67\xda\x87\x82\x92\x00\x00\x00\x00\x49\x45\x4e\x44\xae\x42\x60\x82"

byte_data = base64.b64decode(i)

ValueError: string argument should contain only ASCII characters

In [45]:
x= await search_content_function(mock_request)

12.993813753128052
6.805595874786377
5.481023073196411
4.147134780883789


In [84]:

import functions_framework
import time
import random
from EmbeddingPredictionClient import EmbeddingPredictionClient  
from google.cloud import bigquery
import json
import asyncio
from datetime import datetime


async def exponential_backoff_retries(client, text=None, image_file: bytes=None, max_retries=5, embedding_type=None):
    """
    This function applies exponential backoff with retries to the API calls.
    """
    attempt = 0
    while attempt < max_retries:
        try:
            # Try to get the embedding from the client
            if embedding_type=="multimodal_embedding":                   
                    return client.get_multimodal_embedding("", image_file)            
            elif embedding_type=="text_embedding":
                    return client.get_text_embedding(text)
        except Exception as e:
            attempt += 1
            backoff_delay = min(2 ** attempt + random.uniform(0, 1), 32)  # Exponential backoff with jitter
            print(f"Attempt {attempt} failed with error {e}. Retrying in {backoff_delay:.2f} seconds...")
            time.sleep(backoff_delay)  # Wait before retrying

    raise Exception("Max retries reached. Could not complete the request.")

    
async def generate_query_embedding(client,text=None,image_file=None, embedding_type=None):
    try:
        # Retry logic with exponential backoff to calculate query embeddings
        result = exponential_backoff_retries(embedding_client, text, image_file, embedding_type)
        
        # Respond with the successful embedding response
        return {
            "text_embedding": result.text_embedding,
            "image_embedding": result.image_embedding
        }, 200

    except Exception as e:
        # Handle failure after max retries
        return f"Error: {str(e)}", 500


async def get_media_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50, filter_query=""):
    """Query nearest neighbors using cosine similarity in BigQuery for multimodal embeddings."""
    
    # Record the start time
    start_time = time.time()
    options="""'{"fraction_lists_to_search": 1}'"""
    #options="""'{"use_brute_force":true}' """

    sql = f"""  
         WITH search_results AS
         (
              SELECT
              search_results.base.uri as fileUri,  
              search_results.base.combined_multimodal_id as unique_id,
              search_results.distance,  -- The computed distance (similarity score) between the embeddings
              search_results.base.asset_id ,
              search_results.base.ml_generate_embedding_start_sec as startOffset_seconds,
              search_results.base.ml_generate_embedding_end_sec as endOffset_seconds,  
              search_results.base.content_type as asset_type,
              ROW_NUMBER() OVER (PARTITION BY search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT * FROM  `{dataset}.{table}` WHERE 1=1 {filter_query}), --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                
              ) search_results
              
          ),   
            -- Step 2: Find the minimum distance per asset_id
             ranked_documents AS (
                SELECT
                    asset_id,        
                    MIN(distance) AS min_distance  -- Alternatively, you can use the average distance
                FROM search_results
                GROUP BY asset_id
            )

            -- Step 3: Retrieve the top-k ranked documents based on relevance
            SELECT * FROM (
              SELECT  
                sr.asset_id,  
                sr.fileUri,  
                sr.asset_type,
                rd.min_distance,
                ROW_NUMBER() OVER (PARTITION BY SR.asset_id ORDER BY min_distance ASC) AS IDX,
                STRING_AGG(CONCAT("""+"'{startOffset_seconds:', sr.startOffset_seconds, ',endOffset_seconds:', sr.endOffset_seconds, '}')"""+f""", ", " ) 
                OVER (PARTITION BY sr.asset_id ORDER BY sr.startOffset_seconds) AS time_lines,
                CASE WHEN LOWER(asset_type) like '%video%' then
                 CONCAT(' ',
                   CASE  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)])  LIKE '%NNNT23%' THEN 'NINE NEWS 2023'  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%MAAT2023%' THEN 'MARRIED AT FIRST SIGHT 2023'
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%60MI23%' THEN '60 MINUTES 2023'
                   END ,
                   ' EPISODE ' , UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(1)])
                   )
                 ELSE '' END AS headline
    
               -- sr.distance,
               -- final_rank--,
               -- rank_within_document
            FROM search_results sr
            JOIN ranked_documents rd ON sr.asset_id = rd.asset_id
            ORDER BY min_distance ASC  
            )
            WHERE IDX=1
    """       
    #print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()
    
    output=[]
    for row in results:
        output.append({'asset_id':row['asset_id'], 'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"], "distance":row['min_distance'],
                       "headline":row["headline"]})

    
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

async def get_content_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50,filter_query=""):
    """Query nearest neighbors using cosine similarity in BigQuery for text embeddings."""
    
    # Record the start time
    start_time = time.time()
    options="""'{"fraction_lists_to_search": 1}'"""
    #options="""'{"use_brute_force":true}' """
    
    sql = f"""  
         WITH search_results AS
         (
              SELECT
              search_results.base.content as content,  
              search_results.base.combined_id as combined_id,
              search_results.base.unique_id,
              distance,  -- The computed distance (similarity score) between the embeddings
              search_results.base.asset_id,
              search_results.base.headline,
              ifnull(search_results.base.html_safe_text,search_results.base.description) as description,
              search_results.base.startOffset_seconds,
              search_results.base.endOffset_seconds,
              search_results.base.fileUri,
              search_results.base.asset_type,    
              search_results.base.first_published_timestamp,
              search_results.base.brand_type,
              search_results.base.primary_category_name,
             search_results.base.byline[SAFE_OFFSET(0)].author_name,
              ROW_NUMBER() OVER (PARTITION BY  search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT * FROM  `{dataset}.{table}` WHERE 1=1 {filter_query}), --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                   
              ) search_results              
          ),          

             -- Step 2: Aggregate relevance per document (original_document_id)
            ranked_documents AS (
                SELECT
                    asset_id,        
                    MIN(distance) AS min_distance  -- Alternatively, you can use the average distance
                FROM search_results
                GROUP BY asset_id
            )

            -- Step 4: Retrieve the top-k ranked documents based on relevance
            SELECT * FROM (
              SELECT  
                sr.asset_id, 
                CASE WHEN LOWER(asset_type) like '%video%' then
                 CONCAT(' ',
                   CASE  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)])  LIKE '%NNNT23%' THEN 'NINE NEWS 2023'  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%MAAT2023%' THEN 'MARRIED AT FIRST SIGHT 2023'
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%60MI23%' THEN '60 MINUTES 2023'
                   END ,
                   ' EPISODE ' , UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(1)])
                   )
                 ELSE IFNULL(sr.headline,'') END AS headline,                 
                sr.description,
                sr.combined_id,
                sr.unique_id,
                sr.fileUri,
                sr.asset_type,
                rd.min_distance,
                sr.first_published_timestamp,
                sr.brand_type,
                sr.primary_category_name,
                sr.author_name,
                ROW_NUMBER() OVER (PARTITION BY SR.asset_id ORDER BY min_distance desc) AS IDX,
                STRING_AGG(CONCAT("""+"'{startOffset_seconds:', sr.startOffset_seconds, ',endOffset_seconds:', sr.endOffset_seconds, '}')"""+f""", ", " ) 
                OVER (PARTITION BY sr.asset_id ORDER BY sr.startOffset_seconds) AS time_lines               
                --sr.distance,
                --final_rank--,
               -- rank_within_document
            FROM search_results sr
            JOIN ranked_documents rd ON sr.asset_id = rd.asset_id
            ORDER BY min_distance ASC
            )
            WHERE IDX=1
    """       
 
    #print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()   
    output=[]
    for row in results:      
        output.append({'asset_id':row['asset_id'], 'headline':row['headline'],'description':row['description'],'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"], 
                      "distance":row['min_distance'],
                       "first_published_timestamp":row['first_published_timestamp'].isoformat() if  not row['first_published_timestamp'] is None else row['first_published_timestamp'],
                        "brand_type":row['brand_type'],
                        "primary_category_name":row['primary_category_name'],
                        "author_name":row['author_name']
                      
                      })
  
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

def merge_result(combined_list):
    # Step 2: Create a dictionary to merge by 'id'
    merged_dict = {}

    # Step 3: Iterate through the combined list and merge dictionaries by 'id'
    for d in combined_list:
        id_value = d['asset_id']

        # If the id already exists in merged_dict, update it
        if id_value in merged_dict:
            merged_dict[id_value].update(d)
        else:
            # If the id doesn't exist, add the dictionary as it is
            merged_dict[id_value] = d.copy()

    # Step 4: Convert the merged dictionary back into a list
    final_merged_list = list(merged_dict.values())
    
    return final_merged_list



async def get_nearest_contet(request):
    """
    Cloud Function entry point. This function handles the incoming request, 
    performs exponential backoff retries, and returns the embedding response.
    """ 
    # Parse the incoming request to extract text or image file
    request_json = request.get_json(silent=True)
    if "search_query" in request_json:
        text = request_json.get('search_query')
    else:
        text=""

    if "image_file"  in request_json:
        image_file = request_json.get('image_file')  # Assume it's the path or base64 string of the image
    else:
        image_file=""

    project_id = request_json.get('project')  
    region = request_json.get('region')  

    if "filter_image" in request_json: 
       filter_image = request_json.get('filter_image') 
    else:
        filter_image="True"

    if "filter_video" in request_json:
        filter_video = request_json.get('filter_video') 
    else:
        filter_video="True"
    
    if "filter_article" in request_json:
       filter_article=request_json.get('filter_article')
    else:
        filter_article="True"
    
    # Load configuration from config.json
    with open('config.json') as config_file:
         config = json.load(config_file)
    
    
    top_k=int(config['top_k'])  
    dataset= config['dataset']
    content_table=config['content_table']
    mm_table=config['mm_table']
    content_source_embedding_column=config['content_source_embedding_column']
    mm_source_embedding_column=config['mm_source_embedding_column'] 
    if image_file=="" or image_file=="None":
        image_file=None
        
    article_filter_query=""
    if filter_article=="True" or filter_article=="1":
        article_filter_query= article_filter_query+f" AND lower(asset_type) like '%article%' " 
        
    image_filter_query=""
    if filter_image=="True" or filter_image=="1":
        image_filter_query= image_filter_query+f" AND lower(asset_type) like '%image%' "  
        
    video_filter_query=""
    if filter_video=="True" or filter_video=="1":
        video_filter_query= video_filter_query+f" AND lower(asset_type) like '%video%' "
       

    # Initialize the EmbeddingPredictionClient outside the function for reuse
    embedding_client = EmbeddingPredictionClient(project=project_id , location=region,api_regional_endpoint=region+"-aiplatform.googleapis.com")
        
    if (text=="") and (image_file==None):
        return 'Error: At least one of "text" or "image_file" must be provided.', 400
     
    content_result_article=[]
    content_result_image=[]
    content_result_video=[]
    media_image_result=[]
    error=""
    if 1==1:
        #try:
        if text:
            #if a text is given, calculate both multiomdal embedding and text embedding of the search query
            txtembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='text_embedding'))
            #mmembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='multimodal_embedding')) 
            txtembding_for_text_result=txtembding_for_text_result .text_embedding
            #mmembding_for_text_result=mmembding_for_text_result.text_embedding
            #find nearest neighbours both from text embedding and multimodal embedding
            if article_filter_query!="":
                content_result_article =  asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k, filter_query=article_filter_query))
            if image_filter_query!="":
                content_result_image =  asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k,filter_query=image_filter_query))
            if video_filter_query!="":
                content_result_video =  asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k, filter_query=video_filter_query))
            
            if article_filter_query!="":
                    content_result_article= await(content_result_article)
            if image_filter_query!="":
                    content_result_image=await(content_result_image)
            if video_filter_query!="":
                     content_result_video=await(content_result_video)

            
            
            #media_text_result = await asyncio.create_task(get_media_nearest_neighbors(mmembding_for_text_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k))
                
        if image_file:        
            #if an image is given convert image to 64bytestring and extract embedding
            mmembding_for_image_result = await asyncio.create_task(exponential_backoff_retries(embedding_client, text="",image_file=image_file, embedding_type='multimodal_embedding'))
            mmembding_for_image_result=mmembding_for_image_result.image_embedding
            #find nearest neighbours both from multimodal embedding
            media_image_result = await asyncio.create_task(get_media_nearest_neighbors(mmembding_for_image_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k))
            media_image_result=media_image_result   
    #except Exception as e:
        #error= str(e)

    

    final_merged_list=merge_result(content_result_article+content_result_image+content_result_video+media_image_result)
    #log_data(final_merged_list,error,request_json ,0,project_id)
    return final_merged_list,error, project_id


def log_data(result,error,request,elapsed_time,project_id):
    
    rows_to_insert=[] 
    rows_to_insert.append(
                                {  "search_date":  datetime.now().isoformat() ,
                                    "request":request,
                                    "response":   result  , 
                                    "error":  error,
                                    "elapsed_time":elapsed_time ,
                                    "API": "search"
                                  
                                    }
                                            )

    

    #create table new if does not exist
    # Load configuration from config.json
    with open('config.json') as config_file:
         config = json.load(config_file)

    table=config['log_table']
    dataset_id=config['log_dataset']
    #push the data into the table
    table_id = f"{project_id}.{dataset_id}.{table}"
    client = bigquery.Client(project_id)
    dataset  = client.dataset(dataset_id)
    table = dataset.table(table)
    # job_config = bigquery.LoadJobConfig()
    # job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    # job_config.schema = table_schema
    job = client.load_table_from_json(rows_to_insert, table) 

           
    

@functions_framework.http
async def search_content_function(request):
 
    result = await get_nearest_contet(request) 
    return result#[0],result[1],result[2]

# @functions_framework.http
# def search_content_function(request):
#     """This is the entry point for the Cloud Function."""
#     try:
#         loop = asyncio.get_event_loop()
#     except RuntimeError as e:
#         # If no event loop is running, create a new event loop for this thread
#         loop = asyncio.new_event_loop()
#         asyncio.set_event_loop(loop)
    
#     start_time = time.time()
#     result,error,project_id = loop.run_until_complete(get_nearest_contet(request))
#     end_time = time.time()
#     # Calculate the elapsed time
#     elapsed_time = end_time - start_time

#     #record the search log
#     log_data(result,error,request.get_json(silent=True),elapsed_time,project_id)
#     return result
     

In [85]:
from unittest.mock import Mock
import json
 

# Your input data as a dictionary
data = {"search_query":"trump","image_file":""
,"project":"nine-quality-test","region":"us-central1"}

# Simulating an HTTP request with the mock object
mock_request = Mock()
mock_request.get_json.return_value = data  # Mock the get_json method to return your data


In [86]:
x= await search_content_function(mock_request)

0.6781527996063232
0.608837366104126
0.6286230087280273


In [ ]:
log_data(x,"",mock_request.get_json(silent=True),0.75,'nine-quality-test')

In [87]:
x

([{'asset_id': 'p5cxie',
   'headline': 'Yes, Trump could run for president from prison. One candidate did it in 1920',
   'description': '<p><strong>Washington:</strong> Eugene V. Debs did not speak on election night in 1920. The Socialist presidential contender was, in his words, a “candidate in seclusion,” imprisoned in the Atlanta Federal Penitentiary for speaking out against America’s draft during World War I.</p> <p>Outside the lockup, his supporters handed out photos of Debs in convict denim along with campaign buttons for “Prisoner 9653”. Reporters had hoped to hear a fiery oration. But the warden did let Debs write out a statement.</p> <p>“I thank the capitalist masters for putting me here,” he wrote. “They know where I belong under their criminal and corrupting system. It is the only compliment they could pay me.”</p> <p>In 1920, Debs was no stranger to White House bids; he’d run for president on the Socialist Party ticket five times since 1900. Eight years earlier, he’d won 